In [1]:
# Amend to set wd to STB_social_media_analytics.
%cd /home/jia/Desktop/git/STB_social_media_analytics/experimentation/jiaxin_experiment/ctrip

/home/jia/Desktop/git/STB_social_media_analytics/experimentation/jiaxin_experiment/ctrip


In [2]:
import pandas as pd
import mysql.connector
from scrapy import Selector
import os
import re
import yaml
import utils
from time import sleep
import traceback
from random import random
from selenium import webdriver
from datetime import datetime, date, timedelta
from random import random

In [3]:
with open('config_file_Ctrip.yml') as file:
    configs = yaml.load(file, Loader=yaml.FullLoader)

chromedriver_path = configs['General']['chromedriver_path']

db_in_flag = configs['Ctrip']['db_in_flag']
db_out_flag = configs['Ctrip']['db_out_flag']

if db_in_flag == 'csv':
    
    df = pd.read_csv("ctrip_list.csv")
    
    def parse_reviews(text):
        if re.search(r'\d+',str(text)):
            return int(re.search(r'\d+', text).group())
        else:
            return 0
        
    df['num_reviews'] = df['num_reviews'].apply(parse_reviews)
    df['link'] = df['link'].apply(lambda x: "https://you.ctrip.com"+x)
    
    poi_index=list(range(len(df['POI'])))
    
    poi_df = pd.DataFrame({'poi_index':poi_index, 
                           'poi_name':df['POI'], 
                           'poi_url':df['link']}
                          )
    ### FOR POC ONLY ###    #in crawling, read csv directly
    #poi_index = [1]
    #poi_name = [
               # 'Universal Studio Singapore'
               #]
    #poi_url = [
               #'https://piao.ctrip.com/ticket/dest/t75830.html']
    #poi_df = pd.DataFrame({'poi_index':poi_index, 
                           #'poi_name':poi_name, 
                           #'poi_url':poi_url}
                          #)
    ####################

if db_in_flag in ['sqlite', 'mysql']:
    poi_df = pd.DataFrame()

if db_in_flag in ['sqlite', 'mysql'] or db_out_flag in ['sqlite', 'mysql']:
    cnx = mysql.connector.connect(host=configs['General']['host'],
                                  database=configs['General']['database'],
                                  user=configs['General']['user'],
                                  password=configs['General']['password']
                                 )
else:
    cnx = None
    

#put all hardcoded in to config file

In [4]:
class CrawlCtrip:
    attributes_col_names = ['POI_INDEX',
                            'TOTAL_REVIEWS',
                            'RANKING',
                            'RATINGS',
                            'ADDRESS',
                            'OPENING_HOURS',
                            'ATTRIBUTES_CRAWLED_TIME'
                           ]

    reviews_col_names = ['REVIEW_INDEX',
                         'WEBSITE_INDEX',
                         'POI_INDEX',
                         'REVIEWER_NAME',
                         'REVIEW_RATING',
                         'REVIEW_DATE',
                         'REVIEW_TIME',
                         'REVIEW_BODY',
                         'ATTRIBUTES_CRAWLED_TIME'
                         #'TRANSLATED_REVIEW_BODY_GOOGLE',
                         #'TRANSLATED_REVIEW_BODY_WATSON'
                        ]
    
    
    def __init__(self, chromedriver_path, poi_df, cnx, db_out_flag):
        self.driver = webdriver.Chrome(chromedriver_path)
        self.poi_df = poi_df
        if cnx:
            self.cursor = cnx.cursor()
        self.db_out_flag = db_out_flag
        self.number_of_pages = None
        self.current_page = None
        self.current_poi_index = None
        self.review_final_page = False
        self.attributes_df = pd.DataFrame(columns=self.attributes_col_names)
        self.reviews_df = pd.DataFrame(columns=self.reviews_col_names)
        self.sel= None
        self.current_url=None
        
        # Create unique CSVs.
        self.datetime_string = datetime.now().strftime('%y%m%d_%H%M%S')
        os.makedirs('./output/{}/'.format(self.datetime_string))
        os.makedirs('./output/{}/reviews'.format(self.datetime_string))
        self.attributes_df.to_csv('./output/{}/attributes.csv'.format(self.datetime_string),mode='a', index=False)

                
    
    def add_to_database(self):
        # Read csv, add to database, then cnx.commit().
        return
    
    
    def crawl_pois(self, number_of_pages=None):
        if number_of_pages is not None:
            self.number_of_pages = number_of_pages
        for _, row in self.poi_df.iterrows():
            print('########## {} ##########'.format(row['poi_name']))
        # Create <POI_INDEX>.csv in reviews and reviewers folders.
            self.current_poi_index = row['poi_index']        
            self.reviews_df.to_csv('./output/{}/reviews/{}.csv'.format(self.datetime_string, self.current_poi_index), mode='a', index=False)
            
        # Crawl
            try:
                self.driver.get(row['poi_url'])
    #################trial part to handle error of redirecting########################################
                driver=self.driver
                self.sel = Selector(text=driver.page_source)
                sleep(1+random()*2)
                self. current_url= driver.current_url
                if self.current_url == 'https://you.ctrip.com/':
                    log_file = open('./output/{}/log.txt'.format(self.datetime_string), 'a+')
                    log_file.write('{}, {}, {}, {}'.format(row['poi_index'],
                                                           row['poi_name'],
                                                           datetime.now(),
                                                           'current POI does not exist'))
                    continue
        

    ####################################################################################################
                self.crawl_attributes(row['poi_index'])
                self.attributes_df.to_csv('./output/{}/attributes.csv'.format(self.datetime_string),mode='a', header=False, index=False)
                self.attributes_df = pd.DataFrame(columns=self.attributes_col_names)
                self.crawl_reviews(row['poi_index'])
                if self.db_out_flag != 'csv':
                    self.add_to_database()
       
            except Exception as e:
                log_file = open('./output/{}/log.txt'.format(self.datetime_string), 'a+')
                log_file.write('{}, {}, page: {}, {}\n'.format(row['poi_index'], row['poi_name'], self.current_page, datetime.now()))
                # log_file.write(repr(e) + '\n')
                log_file.write(traceback.format_exc() + '\n')
                log_file.close()
                break
        
                
                
    def crawl_reviews(self, poi_index):
        if self.number_of_pages is not None:
            self.current_page = 1
            for i in range(self.number_of_pages):
                print('Page {}'.format(self.current_page))
                self.crawl_reviews_1_page(poi_index)
                self.reviews_to_csv()
                self.current_page += 1
                if self.review_final_page:
                    self.review_final_page= False
                    break
        else:
            self.current_page = 1
            while not self.review_final_page:
                print('Page {}'.format(self.current_page))
                self.crawl_reviews_1_page(poi_index)
                self.reviews_to_csv()
                self.current_page += 1
                if self.review_final_page:
                    self.review_final_page= False
                    break
                    
                    
    def reviews_to_csv(self):
        self.reviews_df.to_csv('./output/{}/reviews/{}.csv'.format(self.datetime_string, self.current_poi_index), mode='a', header=False, index=False)
        self.reviews_df = pd.DataFrame(columns=self.reviews_col_names)
        
    
    def crawl_attributes(self, poi_index):
        #driver = self.driver
       
        #Forming selector path
        #sel = Selector(text=driver.page_source)

      
        #select areas where we going to extract info
        res = self.sel.xpath('.//div[@class="brief-box clearfix"]')
        res1= self.sel.xpath('.//body') 

        
        if res:
            xpath_total_reviews = '//a[@data-reactid="58"]/text()'    
            total_review =res.xpath(xpath_total_reviews).extract_first() 

            xpath_name = '//h2[@data-reactid="38"]/text()'    
            name=res.xpath(xpath_name).extract_first()   

            xpath_rate = '//i[@class="num"]/text()'    
            rating=res.xpath(xpath_rate).extract_first()  

            xpath_add = '//span[@data-reactid="45"]/text()'    
            address=res.xpath(xpath_add).extract_first()  

            xpath_openhr = '//span[@data-reactid="51"]/text()'   
            opening_hours=res.xpath(xpath_openhr).extract_first()  
            
            # Parsing attributes.
            total_reviews = self.parse_total_reviews1(total_review)
 
        else:
            xpath_total_reviews = '//span[@class="f_orange"]/text()'  
            total_reviews =res1.xpath(xpath_total_reviews).extract_first() 

            xpath_name = '//div[@class="f_left"]/h1/a/text()'    
            name=res1.xpath(xpath_name).extract_first()   

            xpath_rate = '//span[@class="score"]/b/text()'    
            rating=res1.xpath(xpath_rate).extract_first()  

            xpath_add = '//p[@class="s_sight_addr"]/text()' 
            add=res1.xpath(xpath_add).extract_first()  

            xpath_openhr = '//dl[@class="s_sight_in_list"]/dd/text()'    
            opening_hours=res1.xpath(xpath_openhr).extract_first()  
            
            # Parsing attributes
            address=self.parse_attribute_address(add)
            
     
        ranking=poi_index
        #crawltime=(datetime.now()).strftime("%Y-%m-%d %H:%M:%S")
                                    
        poi_attributes = [poi_index,
                          total_reviews, 
                          ranking, 
                          rating,  
                          address, 
                          opening_hours,
                          datetime.now()
                         ]
                                    
        #print(poi_attributes)
        
        # Inserting attributes into dataframe
        poi_attributes_dict = dict(zip(self.attributes_col_names, poi_attributes))
        self.attributes_df = self.attributes_df.append(poi_attributes_dict, ignore_index=True)
        sleep(2+random()*2)

    def crawl_reviews_1_page(self, poi_index):        
        #driver = self.driver
        #sel = Selector(text=driver.page_source)
        sleep(random()*2)
        res = self.sel.xpath('//div[@class="clearfix"]')    
        res1= self.sel.xpath('//div[@class="weibocombox"]')
    
        if res:
            xpath_user_name = './/li/div[@class="user-date"]/span/text()[1]'  
            reviewer_name = res.xpath(xpath_user_name).getall()   

            xpath_review_time ='.//li/div[@class="user-date"]/span/text()[3]'  
            review_time_tgt =res.xpath(xpath_review_time).getall() 
            
            xpath_review='.//li/p/text()'
            review=res.xpath(xpath_review).getall() 
    
            xpath_rates='.//li/h4/span/text()[2]'
            review_ratings=res.xpath(xpath_rates).getall()
            
            #parsing elements
            review_date= list(map(self.parse_review_date1,review_time_tgt))
            review_time= list(map(self.parse_review_time1,review_time_tgt))
            review_rating= list(map(self.parse_review_rating1,review_ratings))
            review_body=list(map(self.parse_review_body1,review))
        
        else:
            xpath_user_name = './/a[@itemprop="author"]/text()'  
            reviewer_name = res1.xpath(xpath_user_name).getall()   

            xpath_review_time ='.//em[@itemprop="datePublished"]/text()'  
            review_date =res1.xpath(xpath_review_time).getall()   

            xpath_review='.//span[@class="heightbox"]/text()'
            review_body1=res1.xpath(xpath_review).getall() 
    
            xpath_rates='.//ul//span/@style'
            rates=res1.xpath(xpath_rates).getall()
            
            review_time = 'Not Available'
            
            # Parsing attributes.
            review_ratings= list(map(self.parse_review_rating2,rates))
            review_body=list(map(self.parse_review_body1,review_body1))
        
        
        
        review_details = [None, # REVIEW_INDEX
                          2, # WEBSITE_INDEX (Ctrip is '2')
                          poi_index,
                          reviewer_name,
                          review_ratings,
                          review_date,
                          review_time,
                          review_body,
                          datetime.now()
                          ]
            
            
            
        # Inserting reviews into dataframe.
        review_details_dict = dict(zip(self.reviews_col_names, review_details))
        self.reviews_df = self.reviews_df.append(review_details_dict, ignore_index=True)
        
        if res:
            next_page_button = self.driver.find_elements_by_xpath('//a[@class="down  "]')
            if next_page_button:
                sleep(1+random()*2)
                next_page_button[0].click()
                sleep(1+random()*2)
                self.sel = Selector(text=self.driver.page_source)
            else:
                print("Can't click or last page")
                self.review_final_page= True
                    
        else:
            next_page_button = self.driver.find_elements_by_xpath('//a[@class="nextpage"]')
            if next_page_button:
                sleep(1+random()*2)
                self.driver.execute_script("arguments[0].click()",next_page_button[0])
                sleep(1+random()*2)
                self.sel = Selector(text=self.driver.page_source)
            else:
                print("Can't click or last page")
                self.review_final_page= True
            
        
            
        

    
    # Methods below are all utility functions.
    def parse_total_reviews1(self, text):
        if re.search(r'\d+',str(text)):
            return int(re.search(r'\d+', text).group())
        else:
            return 0
    
    def parse_attribute_address(self, text):
        return (text[3:]).replace(' ','')
    
    def parse_review_date1(self, text):
        return datetime.strptime(text[0:10],"%Y-%m-%d")
        
    def parse_review_time1(self, text):
        return datetime.strptime(text[11:16],"%H:%M")

    
    def parse_review_rating1(self, text):
        return int(text)
    
    def parse_review_rating2 (self, text):
        return int(text[6:text.find('%')])/20
    
    def parse_review_body1(self, text):
        return text.replace('\n','')

    

In [6]:
CrawlCtrip(chromedriver_path, poi_df, cnx, db_out_flag).crawl_pois(number_of_pages=5)

########## 新加坡环球影城 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 新加坡滨海湾花园 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 圣淘沙岛 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 乌节路 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 克拉码头 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 新加坡动物园 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 小印度 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 牛车水 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 新加坡摩天观景轮 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## S.E.A.海洋馆 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 鱼尾狮公园 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 夜间野生动物园 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 滨海湾金沙 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 新加坡植物园 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 圣淘沙名胜世界 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 滨海艺术中心 ##########
Page 1
Page 2
Pa

Page 1
Page 2
Page 3
Page 4
Page 5
Can't click or last page
########## 圣淘沙4D探险乐园 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 孙中山南洋纪念馆 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 知新馆 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## The Substation ##########
Page 1
Page 2
Page 3
Can't click or last page
########## 万象馆 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 新加坡总统府 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 圣淘沙大自然探索中心 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 珊顿大道 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
Can't click or last page
########## 动感电影院 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
Can't click or last page
########## 樟宜村 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
Can't click or last page
########## 阿都卡夫清真寺 ##########
Page 1
Page 2
Page 3
Page 4
Page 5
########## 南部山脊 ##########
Page 1
Page 2
Page 3
Can't click or last page
########## 樟宜机场Canopy游乐场 ##########
Page 1
Page 2
Page 3
Page 4
Can't c

Page 1
Page 2
Page 3
Can't click or last page
########## The Cathay Gallery ##########
Page 1
Page 2
Can't click or last page
########## 美年径室内滑雪课程 ##########
Page 1
Can't click or last page
########## XD剧院 ##########
Page 1
Page 2
Can't click or last page
########## 警察文物中心 ##########
Page 1
Page 2
Can't click or last page
########## Yueh Hai Ching Temple ##########
Page 1
Page 2
Can't click or last page
########## 新加坡超级跳 ##########
Page 1
Page 2
Can't click or last page
########## 马达加斯加：木箱漂流记(马达加斯加) ##########
Page 1
Page 2
Page 3
Can't click or last page
########## Lazarus码头私人游艇出海 ##########
Page 1
Can't click or last page
########## 三巴旺公园 ##########
Page 1
Page 2
Page 3
Page 4
Can't click or last page
########## 新加坡军事博物馆 ##########
Page 1
Page 2
Can't click or last page
########## 乌敏岛皮划艇体验 ##########
Page 1
Can't click or last page
########## Opera Gallery ##########
Page 1
Page 2
Can't click or last page
########## Istana Woodneuk ##########
Page 1
Page 2
Can't click or last page
##

Page 1
Can't click or last page
########## Monster Day Tours ##########
Page 1
Can't click or last page
########## Puay Heng Noodles ##########
Page 1
Can't click or last page
########## Church of St Teresa ##########
Page 1
Can't click or last page
########## Tourist and Transit Hub ##########
Page 1
Page 2
Can't click or last page
########## Coney Island Park ##########
Page 1
Can't click or last page
########## Sri Layan Sithi Vinayagar Temple ##########
Page 1
Page 2
Can't click or last page
########## 新加坡通勤馆  ##########
Page 1
Can't click or last page
########## Southernmost point of Asia ##########
Page 1
Can't click or last page
########## Marine Cove Playground ##########
Page 1
Can't click or last page
########## Butterfly Garden ##########
Page 1
Can't click or last page
########## Oh Chin Huat Hydroponic Farms ##########
Page 1
Can't click or last page
########## 仙祖宫 ##########
Page 1
Can't click or last page
########## 武吉巴督自然公园 ##########
Page 1
Page 2
Can't click or last p

Can't click or last page
########## 空军博物馆 ##########
Page 1
Page 2
Can't click or last page
########## 嘉华电影 ##########
Page 1
Can't click or last page
########## 医学院大楼 ##########
Page 1
Can't click or last page
########## Singapore Dance Theatre ##########
Page 1
Can't click or last page
########## Shen Nong Spa ##########
Page 1
Can't click or last page
########## 中央锡克庙 ##########
Page 1
Page 2
Can't click or last page
########## Kim Seng Park ##########
Page 1
Can't click or last page
########## Tampines Changkat Butterfly Garden ##########
Page 1
Can't click or last page
########## Katong Park ##########
Page 1
Can't click or last page
########## 長老會恩澤堂裕廊聚會 ##########
Page 1
Can't click or last page
########## 新加坡采石场 ##########
Page 1
Page 2
Can't click or last page
########## Tang Dynasty Massage & Spa ##########
Page 1
Can't click or last page
########## Traditional Oriental Therapy ##########
Page 1
Can't click or last page
########## Galerie Belvedere ##########
Page 1
Can't cli

...

Page 1
Can't click or last page
########## British Airways Concorde Bar ##########
Page 1
Can't click or last page
########## Al Ansar Mosque ##########
Page 1
Can't click or last page
########## Seventh Day Adventist Church - Jurong East ##########
Page 1
Can't click or last page
########## Paya Lebar Methodist Church ##########
Page 1
Can't click or last page
########## Limah pin pat school ##########
Page 1
Can't click or last page
########## Assyakirin Mosque ##########
Page 1
Can't click or last page
########## Choa Chu Kang Park ##########
Page 1
Can't click or last page
########## Danish Seamen's Church ##########
Page 1
Can't click or last page
########## Lantern ##########
Page 1
Can't click or last page
########## British Airways Concorde Bar ##########
Page 1
Can't click or last page
########## Cin Cin ##########
Page 1
Can't click or last page
########## Paya Lebar Methodist Church ##########
Page 1
Can't click or last page
########## Eu Chin Street ##########
Page 1
Can't 

Page 1
Can't click or last page
########## Amore Fitness & Boutique Spa ##########
Page 1
Can't click or last page
########## The Joy Of Toys ##########
Page 1
Can't click or last page
########## G-MAX逆转蹦极 ##########
Page 1
Can't click or last page
########## Amore Fitness & Boutique Spa ##########
Page 1
Can't click or last page
########## Sultan Gate ##########
Page 1
Can't click or last page
########## Osteopathy & Podiatry Centre ##########
Page 1
Can't click or last page
########## LE Privaté Clinic ##########
Page 1
Can't click or last page
########## Lim & Chin Ventures ##########
Page 1
Can't click or last page
########## Joo Chiat Beef King ##########
Page 1
Can't click or last page
########## Jin Hua Sliced Fish Bee Hoon ##########
Page 1
Can't click or last page
########## Tien Hong Vegetable Rice Porridge ##########
Page 1
Can't click or last page
########## Rong Ji Mei Shi ##########
Page 1
Can't click or last page
########## Tai Feng (Handmade) Fish Ball noodle ##########

Page 1
Can't click or last page
########## World of Birdnest Museum ##########
Page 1
Can't click or last page
########## Nithya Priyan School of Yoga ##########
Page 1
Can't click or last page
########## Bike Around Tour ##########
Page 1
Can't click or last page
########## Ang Chee Sia Ong Temple ##########
Page 1
Can't click or last page
########## Church of St. Alphonsus ##########
Page 1
Can't click or last page
########## Tiong Bahru Air Raid Shelter ##########
Page 1
Can't click or last page
########## Sky Pilates ##########
Page 1
Can't click or last page
########## Singapore Pools ##########
Page 1
Can't click or last page
########## Scuba Diving ##########
Page 1
Can't click or last page
########## LKY School of Public Policy ##########
Page 1
Can't click or last page
########## Absolute Zero - Scotts Road ##########
Page 1
Can't click or last page
########## Crazy Elephant ##########
Page 1
Can't click or last page
########## DYMK ##########
Page 1
Can't click or last page
#

Page 1
Can't click or last page
########## Amore Fitness & Boutique Spa ##########
Page 1
Can't click or last page
########## Saint George's Church ##########
Page 1
Can't click or last page
########## Covenant Vision Centre ##########
Page 1
Can't click or last page
########## Archbishop's House ##########
Page 1
Can't click or last page
########## the Ridges ##########
Page 1
Can't click or last page
########## Lovers Bridge ##########
Page 1
Can't click or last page
########## Statue of Sir Stamford Raffles ##########
Page 1
Can't click or last page
########## Juniors' Sailing Club ##########
Page 1
Can't click or last page
########## Wego.com ##########
Page 1
Can't click or last page
########## Filmgarde ##########
Page 1
Can't click or last page
########## Tantric Bar ##########
Page 1
Can't click or last page
########## Iwannagohome! ##########
Page 1
Can't click or last page
########## Lantern at Fullerton Bay Hotel ##########
Page 1
Can't click or last page
########## Church O

Page 1
Can't click or last page
########## Beauty 21 ##########
Page 1
Can't click or last page
########## DBS Arts Centre ##########
Page 1
Can't click or last page
########## Green Apple Spa ##########
Page 1
Can't click or last page
########## TC Reflexology ##########
Page 1
Can't click or last page
########## Away Spa ##########
Page 1
Can't click or last page
########## Castelle de Beaute ##########
Page 1
Can't click or last page
########## 甘露保健SPA ##########
Page 1
Can't click or last page
########## i Spa ##########
Page 1
Can't click or last page
########## Han Dynasty Spa ##########
Page 1
Can't click or last page
########## The Imperial Massage & Spa ##########
Page 1
Can't click or last page
########## It works ##########
Page 1
Can't click or last page
########## 甘露保健SPA ##########
Page 1
Can't click or last page
########## Ikeda Spa ##########
Page 1
Can't click or last page
########## Green Apple Spa ##########
Page 1
Can't click or last page
########## i Spa ##########